In [61]:
%matplotlib inline
from scipy.io import loadmat
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
import numpy as np
import os
import random
import csv

print('done')

done


<h2> Process Multiple Files</h3>

<h3>Make F0 raw csv files per phoneme

In [62]:
# time step is 0.005 seconds, but the syll time resolution is 4dp, so must round

# for each mat file:
#     load mat file
#     extract f0_raw, syll_label and syll_time into variables, and reshape
#     for each syll_label
#         if syll_label contains a '#' in it
#             get syll_time start and end times
#             divide each by 0.005 to get start and end indexes
#             extract f0_raw range using indexes
#             do the f0 rounding, shifting, casting as before
#             write to file, adding syllable loop index to end of filename
#         else
#             skip to next syllable

# must also modify combo source/target section that follows, to remove segmenting parts

# ###########################################

# set number of decimal places
dec = 0

# path to input files directory
directory_path_root = '/Users/robinson/Dropbox/anasynth/_data/emoVC/Olivia2006'
directory = os.fsencode(directory_path_root)
# path to output files directory
directory_path_f0raw = os.path.join(directory_path_root, 'f0_raw_phoneme')
if not os.path.exists(directory_path_f0raw):
    os.mkdir(directory_path_f0raw)

    
# lists to store all syllables and phonemes in all files
all_phonemes = []
all_syllables = []

# list of unvoiced phoneme labels
vowel_phonemes = ['e~', '9~', 'a~', 'o~', 'i', 'e', 'E', 'a', 'A', 'O', 'o', 'u', 'y', '2', '9', '@']

# prefix to add to f0 values based on syllable position in phrase
prefix = ''

def max_round_save(i, k, f0_accum_list, phone_label_string):
    # impose max f0 value
    max_f0 = 550
    # voiced = [True if '#' not in syll[0] else False for i, syll in enumerate(syll_label)]\n",
    f0_accum_list = [x if x < max_f0 else max_f0 for x in f0_accum_list]
    f0_accum_array = np.array(f0_accum_list)
    
    # create new array to hold rounded values
    phone_f0_dec = np.zeros(f0_accum_array.shape)
    # round all values to dec dp
    np.around(f0_accum_array, decimals=dec, out=phone_f0_dec)
    # multiply by 10^dec to shift dp dec places to the right
    phone_f0_dec = phone_f0_dec * (10**dec)
    # cast to int to ensure precise number representation in memory
    phone_f0_dec = phone_f0_dec.astype(int)
    
    # comment out for no con
    phone_f0_dec_prefixed = []
    [phone_f0_dec_prefixed.append(prefix + str(x)) for x in phone_f0_dec]
    
    # write out csv file of f0_raw values - specify format as %u for values to be written as int
    # add syllable loop index to end of filename
    filename_noext, _ = os.path.splitext(filename)
    output_file_extension = '.csv'
    # removing the syllable marker, as the same phoneme may not belong to the same syllable
    # output_file_name = ''.join([filename_noext, '.s', format(i, '02d'), 
    #                             '_', phone_label_string, str(k), output_file_extension])    
    output_file_name = ''.join([filename_noext, '_', phone_label_string, str(k), output_file_extension])
    # print('output_file_name: ', output_file_name)
    np.savetxt(os.path.join(directory_path_f0raw, output_file_name), phone_f0_dec_prefixed, delimiter=',', fmt='%s')                         


# for each mat file in directory (each mat file has one sequence of f0 raw values in it)
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith('.mat'): 
        # build filepath (should use file var here really)
        filepath = os.path.join(directory_path_root, filename)
        # print(filepath)
        
        # load the file and extract f0 raw, syll_label and syll_time into variables, and reshape
        mat_dict = loadmat(filepath)
        f0_raw = mat_dict['f0_raw']
        f0_raw = f0_raw.reshape((f0_raw.shape[1],))
        
        syll_label = mat_dict['syll_label']
        syll_label = syll_label.reshape((syll_label.shape[1],))        
        # print(syll_label.shape)
        # print(syll_label)

#         for label in syll_label:
#             print(label[0])
        
        # reshape this to 2d, to preserve start/end relationship
        # syll_time.shape (2, 11)
        # I want syll_time.shape (11, 2) BUT with the start and end times in different 'columns' - just transpose!
        syll_time = mat_dict['syll_time']
        # print('syll_time.shape', syll_time.shape)
        syll_time = syll_time.T
        # print('syll_time.shape', syll_time.shape)
        # print(syll_time)
        
        first_nonhash_index = 999
        last_nonhash_index = 999
        
        # get first non-hash syllable index
        for i, syll in enumerate(syll_label):
            if '#' not in syll[0]:
                first_nonhash_index = i
                break
        # print('first_nonhash_index', first_nonhash_index)
        
        # get last non-hash syllable index
        indicies = reversed(range(len(syll_label)))
        for i, syll in zip(indicies, syll_label):
            if '#' not in syll[0]:
                last_nonhash_index = i
                break
        # print('last_nonhash_index', last_nonhash_index)
                
        # get phoneme labels and time
        phone_label = mat_dict['phone_label']
        phone_label = phone_label.reshape((phone_label.shape[1],))
        # [print('phone_label ' + str(x) + ':', phone_label[x][0]) for x in range(phone_label.shape[0])]
        phone_time = mat_dict['phone_time']
        phone_time = phone_time.T

        # break #debug
        
        # for each syll
        # if # skip
        # else get list of phonemes in that syllable - how? the phone time and syll times align perfectly, so search        
        # iterate through phonemes
        #     if unvoiced reached (lookup in list of unvoiced)
        #           if f0 list not empty
        #               impose max/round/write out, using accumulated phone labels as filename
        #               reset phone_label_list
        #           continue iteration
        #     else voiced
        #           get start/end times, calc indices and extract f0 for it, add to accumulating list (also add 
        #           phoneme label to string)
        #     if end of list reached (use else block on loop)
        #           impose max/round/write out, and break to next syllable
        #           reset phone_label_list
        
        # i'll need to modify f0_transform too, to calc the start/end times using the durations of the phonemes in 
        # the mat file 
        
        # dict to hold phoneme identifiers for this file (phrase)
        phon_dict = {}        

        # for each syll in syll_label
        for i, syll in enumerate(syll_label):

            # if syll_label doesn't contain a '#' in it
            if '#' not in syll[0]:
                # print(syll[0])
                
                # if first syllable, prefix with 'a'
                if i == first_nonhash_index:
                    prefix = 'a'
                # if last syllable, prefix with 'c'
                elif i == last_nonhash_index:
                    prefix = 'c'
                # if anything else, it's a middle syllable, prefix with 'b'
                else:
                    prefix = 'b'
                

                # add syllable to a list
                all_syllables.append(syll[0] + str(i))

                # get syll_time start and end times
                # print('i = ', i)
                syll_start_time = syll_time[i, 0]
                syll_end_time = syll_time[i, 1]
                
                phone_id_list = []
                # for each phone label in phone_label (this mat file)
                for j, label in enumerate(phone_label):
                    # get start/end times from phone_time
                    phone_start_time = phone_time[j, 0]
                    phone_end_time = phone_time[j, 1]
                    # if start time => syll_start_time AND end time <= syll_end_time then this phoneme is in the syllable
                    if phone_start_time >= syll_start_time and phone_end_time <= syll_end_time:
                        # add phoneme label id to a list (to use to reference label and start/end times)
                        phone_id_list.append(j)
                
                # print('phone_id_list ', phone_id_list)
                
                # iterate through phonemes of this syllable
                for k in phone_id_list:
                    # make dictionary key for phrase/phoneme combo
                    # keys and vals can be of any type
                    # use 'in' to check if a key is in a dict (but not a value)
                    
                    # Do a fresh dict for each mat file. For each phoneme, add the phon_label to the dict as a key 
                    # with value <phon_label>_1 (f for phoneme). if it already exists, increment the value to 
                    # 2.. Use this value as the postfix on the filename. As long as there are the same number of 
                    # phonemes in each file, regardless of their position, they will match up during the 
                    # intersection.   
                    
                    # check if phone_label[k][0] is in phon_dict
                    # if so, incremement the value and save value into phon_incval
                    if phone_label[k][0] in phon_dict:
                        phon_dict[phone_label[k][0]] = phon_dict[phone_label[k][0]] + 1
                        phon_incval = phon_dict[phone_label[k][0]]
                        # print('IN')
                    # if not, add it with value 1 and save value into phon_incval
                    else:    
                        phon_dict[phone_label[k][0]] = 1
                        phon_incval = phon_dict[phone_label[k][0]]
                        # print('NOT')
                    # append the phone_label[k][0] + phon_incval to all_phonemes 
                    # print('phone_label[k][0] = ', phone_label[k][0])
                    # print('phon_label = {}'.format(phone_label[k][0] + str(phon_incval)))
                    all_phonemes.append(phone_label[k][0] + str(phon_incval))
                    # all_phonemes.append(phone_label[k][0] + str(k))
                    # print('all_phonemes.append : ', phone_label[k][0] + str(k))
                    
                    # reset vars
                    f0_accum_list = []       
                    # if unvoiced reached (lookup in list of unvoiced) if f0 list not empty..
                    # i.e. if you hit a non-vowel but there's nothing in f0_accum_list, then just skip ahead
                    # if phone_label[k][0] not in vowel_phonemes and f0_accum_list:
                    #     print('IN NON-VOWEL PHONEME')
                    #     print('k : ', k)
                    #     print('non-vowel: ', phone_label[k][0])
                    #     # impose max/round/write out, using accumulated phone labels
                    #     # MUST use k-1 as we're on the NEXT phoneme, but want to write prev
                    #     # e.g. s O l, on the l but write O
                    #     print('writing previous phoneme... ', phone_label[k-1][0])
                    #     max_round_save(i, k - 1)
                    if phone_label[k][0] not in vowel_phonemes:
                        # do nothing
                        # print('IN NON-VOWEL PHONEME (f0 list empty)')
                        # print('k : ', k)
                        # print('non-vowel: ', phone_label[k][0])
                        continue
                    # vowel, so accumulate f0 values
                    else:
                        # print('IN VOWEL PHONEME')
                        # print('k : ', k)
                        # print('vowel: ', phone_label[k][0])
                        # get start/end times, calc indices and extract f0 for it
                        phone_start_time = phone_time[k, 0]
                        phone_end_time = phone_time[k, 1]                        

                        # divide each by 0.005 to get start and end indexes
                        phone_start_idx = (int)(phone_start_time // 0.005)
                        phone_end_idx = (int)(phone_end_time // 0.005)
                        # print(phone_start_idx)
                        # print(phone_end_idx)
        
                        # extract f0_raw range using indexes
                        phone_f0 = f0_raw[phone_start_idx:phone_end_idx]
                        # print('phone_f0 ', phone_f0)
                        
                        # add to accumulating list
                        f0_accum_list = list(phone_f0)

                        phone_label_string = phone_label[k][0]
                        
                        # print('phone_label_string', phone_label_string)
                        
                        max_round_save(i, phon_incval, f0_accum_list, phone_label_string)
                        
                # else:
                #     print('IN FINAL ELSE - NO MORE PHONEMES IN SYLL')
                #     # if there are remaining values in f0 list
                #     if f0_accum_list:
                #         print('k OUT of for: ', k)
                #         # impose max/round/write out, using accumulated phone labels as
                #         max_round_save(i, k)
#                 break #debug

            # syll_label contains a '#' in it (an unvoiced region), skip to next syllable
            else:
                continue
                
        # break  # debug - one iteration only
        
    # file is not .mat, so skip
    else:
        continue
        
print('done')

done


<h3> Make Combo Source and Target Syllable Input Files</h3>

In [63]:
# the above code makes output files with one row per syllable
# we can't use these as-is, as we need to read the lines in as pairs, so source and target must have equal num of rows
# next step is to pair the files using the phrase and intensities in the filenames

# source: 10 phrases of i00 intensity across e01 to e08 - each phrase is said 8 times, neutrally
# target: 10 phrases of i01-i05 intensity for e02 - each phrase is said 5 times, expressively (5 times)
# so for each utterance (8 of) of each 'p' source phrase (10 of), copy it 5 times, matched with i01-i05 of 'p' target
# P(10) > E(8) > I(5)

# build paths and open output files
# path to input files directories
input_directory_path = '/Users/robinson/Dropbox/anasynth/_data/emoVC/Olivia2006/f0_raw_phoneme'
# define filename components
# Olivia2006.e02.p01.i01.csv
input_file_root = 'Olivia2006'
input_file_extension = '.csv'

# define output filenames and paths
output_directory = os.path.join(input_directory_path, 'out')
if not os.path.exists(output_directory):
    os.mkdir(output_directory)
# output filenames
filename_source = 'source.txt'
filename_target = 'target.txt'
filename_log = 'log.txt'
# open output files in subdirectory of input files directory (must create manually)
fs = open(os.path.join(output_directory, filename_source), 'w')
ft = open(os.path.join(output_directory, filename_target), 'w')
fo = open(os.path.join(output_directory, filename_log), 'w')


# pass it a symbol string 'p' / 'e' / 'i' with range, or a phoneme code string
# it finds all files in a directory that have this in their filename, and returns their filenames as a set
def getSet(symbol, num_from=None, num_to=None):
    # path to input files directory
    directory = os.fsencode(input_directory_path)
    
    # filepath_list = []
    filename_list = []
    
    # for each csv file in directory (each csv file has one sequence of f0 raw values in it)
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith('.csv'): 
            # build filepath (should use file var here really)
            # filepath = os.path.join(input_directory_path, filename)
            
            # if num_from is set, then it's either a p/e/i, so loop the range specified
            if num_from != None:
                for i in range(num_from, num_to + 1):
                    if ''.join(['.', symbol, format(i, '02d')]) in filename:
                        filename_list.append(filename)
            # if num_from is not set, then it's a phoneme symbol specified
            else:
                if ''.join(['_', symbol, '.']) in filename:
                        filename_list.append(filename)

    # return a set of unique filenames that satisfy the given parameters
    return set(filename_list)


# #####################
# DEFINE PARAMETERS

# 1. Joie douce
# 2. Joie explosive
# 3. Peur contenue
# 4. Peur hystérique
# 5. Tristesse contenue
# 6. Tristesse larmoyante
# 7. Colère contenue
# 8. Colère explosive

# define phrase range
phrase_from = 1
phrase_to = 10
# phrase_list = [1, 3, 5, 10]
phrase_list = [2, 4, 6, 7, 8, 9]
# define source and target emotion ranges
source_emotion_from = 1
source_emotion_to = 8
target_emotion_from = 8  # 2 for joie explosif, 8 for colere explosif
target_emotion_to = 8
# define source and target intensity ranges
source_intensity_from = 0
source_intensity_to = 0
target_intensity_from = 1
target_intensity_to = 5

# END PARAMETERS
# #######################

# SOURCE
# create lists of sets for each phrase, emotion, intensity and phoneme code
set_source_emotions = getSet('e', source_emotion_from, source_emotion_to)
set_target_emotions = getSet('e', target_emotion_from, target_emotion_to)
set_source_intensities = getSet('i', source_intensity_from, source_intensity_to)
set_target_intensities = getSet('i', target_intensity_from, target_intensity_to)

# print(set_source_intensities)

# that do too, then I just  - do this for all syllables for each syllable, get set of source filenames 
# which satisfy the parameters, and a set of target filenames that do too, then I just make a set of filename pairs 
# with a loop (for each filename in source set, match with a filename in target set) - do this for all syllables

# get unique list of syllables
all_phonemes_set = set([x for x in all_phonemes])
# print(len(set_one_syllable))
# print(all_syllables_set)

# NEW pseudo code
# for each phrase in specified phrase range
# for phrase in range(phrase_from, phrase_to + 1):
for phrase in phrase_list:    
#     get set for that phrase (e.g. getSet('p',1,1))
    set_phrases = getSet('p', phrase, phrase)
    # create empty list to store syllables in the phrase
    phrase_phonemes = []

    # get all syllables in that phrase
    # for each syllable
    for phon in all_phonemes_set:
        # get list of filenames that have this syllable        
        set_one_phoneme = getSet(phon)
        # if this syllable exists in this set of phrases, add to the list
        if set_one_phoneme & set_phrases:
            phrase_phonemes.append(phon)
    
    # make a set to remove duplicates
    set_phrase_phonemes = set(phrase_phonemes)

    # for each syllable in this phrase
    for phon in set_phrase_phonemes:
        # get list of filenames that have this phoneme        
        set_one_phoneme = getSet(phon)

        # get source filenames for that one phrase, one phon, all emotions combo
        # - note: returned set can be empty
        if set_one_phoneme & set_phrases & set_source_emotions & set_source_intensities:
            set_sources = set.intersection(set_one_phoneme, set_phrases, set_source_emotions, set_source_intensities)
    #         print(set_sources)
        else:
            continue

        # get target filenames for that one phrase, one phon, all emotions combo
        if set_one_phoneme & set_phrases & set_target_emotions & set_target_intensities:
            set_targets = set.intersection(set_one_phoneme, set_phrases, set_target_emotions, set_target_intensities)
    #         print(set_targets)
        else:
            continue

        # make pairs of all source and target filenames and write out
        # make a set of filename pairs (for every filename in source set, match with every filename in target set)
        for source_file in set_sources:
            for target_file in set_targets:

                # build the source file path
                source_file_path = os.path.join(input_directory_path, source_file)           
                # build the target file path
                target_file_path = os.path.join(input_directory_path, target_file)
                # if this file doesn't exist, break out of syllable loop and try next one
                if not os.path.isfile(target_file_path) or os.stat(target_file_path).st_size == 0 or not \
                        os.path.isfile(source_file_path) or os.stat(source_file_path).st_size == 0:
                    break

                # load the source file and extract vars
                # source_f0_raw = np.loadtxt(source_file_path, dtype=np.dtype('S2'))
                source_f0_raw = np.loadtxt(source_file_path, dtype='str')
                # print(source_f0_raw) ['b176' 'b178' 'b180' 'b183' 'b185' 'b187' 'b189' 'b190' 'b191' 'b192'
                #  'b194' 'b196' 'b199' 'b202' 'b206' 'b209' 'b212' 'b214' 'b216' 'b217'
                #  'b217']
                # reshape to have two indices, the first being a constant so all values belong to the same 'row'
                # source_f0_raw = source_f0_raw.reshape((1, source_f0_raw.shape[0]))
                # print(source_f0_raw.shape)
                # print(source_f0_raw)
                # append it to output file as a new row, with space delimiter between elements, format unsigned int
                # print('source_f0_raw.dtype ', source_f0_raw.dtype)
                # print('source_f0_raw.dtype[0] ', source_f0_raw[0].dtype)
                csvWriterS = csv.writer(fs, delimiter=' ')  # using the csv module to write the file
                csvWriterS.writerow(source_f0_raw)                 # write every row in the matrix
                # np.savetxt(fs, source_f0_raw, fmt='%s', delimiter=' ')

                # load the target file and extract vars
                target_f0_raw = np.loadtxt(target_file_path, dtype='str')
                # reshape to have two indices, the first being a constant so all values belong to the same 'row'
                # target_f0_raw = target_f0_raw.reshape((1, target_f0_raw.shape[0]))                    
                # append it to output file as a new row, with space delimiter between elements, format unsigned int
                csvWriterT = csv.writer(ft, delimiter=' ')  # using the csv module to write the file
                csvWriterT.writerow(target_f0_raw)                 # write every row in the matrix
                # np.savetxt(ft, target_f0_raw, fmt='%s', delimiter=' ')

                # write input and output file pair to log file
                logstring = source_file_path + '   ' + target_file_path
                print(logstring, file=fo)
                
# close the output files
fs.close()
ft.close()
fo.close()

print('done')

done


In [64]:
# shuffle the source/target pairs and split them out into train/val/test files

# set ratios for train/val/test split e.g. 0.6, 0.2, 0.2
train_split = 0.85
val_split = 0.15
test_split = 0.0  # ok for this to be 0.0, but not the others
shuffle = False
# 
# # open source and target input files to read from
# fs = open(os.path.join(output_directory, filename_source), 'r')
# ft = open(os.path.join(output_directory, filename_target), 'r')

# get line counts of files (source and target will be the same, so just need to check one of them)
with open(os.path.join(output_directory, filename_source)) as f:
    f_lines = sum(1 for _ in f)
    # set index values for train, val and test
    train_lines = int(f_lines // (1 / train_split))
    val_lines = int(f_lines // (1 / val_split))
    test_lines = f_lines - train_lines - val_lines # whatever is left

# double check that source and target have the same number of lines
with open(os.path.join(output_directory, filename_target)) as f2:
    f_lines2 = sum(1 for _ in f2)
    if f_lines != f_lines2:
        raise ValueError('Not the same')
    
# open source and target input files to read from
fs = open(os.path.join(output_directory, filename_source), 'r')
ft = open(os.path.join(output_directory, filename_target), 'r')

# read the source and target input files line by line, stripping all whitespace and empty lines
source_data = fs.read().strip().split('\n')
# print(type(source_data))
# print(len(source_data)) #6597
target_data = ft.read().strip().split('\n')
# print(len(target_data)) #6597

# make a list of tuples, each holding a pair of source and target strings
merged_data = list(zip(source_data, target_data))
# shuffle the tuples (preserving the pairing) to ensure a good mix of p/e/i in each set
if shuffle:
    random.shuffle(merged_data)
# print(len(merged_data)) #6597

# seperate the tuples into two lists of source and target lines
train_data_source = [x[0] for x in merged_data[:train_lines]]
train_data_target = [x[1] for x in merged_data[:train_lines]]
val_data_source = [x[0] for x in merged_data[train_lines:(train_lines+val_lines)]]
val_data_target = [x[1] for x in merged_data[train_lines:(train_lines+val_lines)]]
test_data_source = [x[0] for x in merged_data[(train_lines+val_lines):]]
test_data_target = [x[1] for x in merged_data[(train_lines+val_lines):]]

print(len(train_data_source))
print(len(train_data_target))
# print(len(val_data_source))
# print(len(val_data_target))
# print(len(test_data_source))
# print(len(test_data_target))

# make train, test, dev, model directories
train_dir = os.path.join(output_directory, 'train')
dev_dir = os.path.join(output_directory, 'dev')
test_dir = os.path.join(output_directory, 'test')
model_dir = os.path.join(output_directory, 'model')
if not os.path.exists(train_dir):
    os.mkdir(train_dir)
if not os.path.exists(dev_dir):
    os.mkdir(dev_dir)
if not os.path.exists(test_dir):
    os.mkdir(test_dir)
if not os.path.exists(model_dir):
    os.mkdir(model_dir)
    
# open output files to write to
f_train_source = open(os.path.join(train_dir, 'train_source.txt'), 'w')
f_train_target = open(os.path.join(train_dir, 'train_target.txt'), 'w')
f_val_source = open(os.path.join(dev_dir, 'val_source.txt'), 'w')
f_val_target = open(os.path.join(dev_dir, 'val_target.txt'), 'w')
f_test_source = open(os.path.join(test_dir, 'test_source.txt'), 'w')
f_test_target = open(os.path.join(test_dir, 'test_target.txt'), 'w')

# print(train_data_source)

# write each of the lists to the opened files
print(len([line for line in train_data_source]))
print(len([line for line in train_data_target]))

[print(line, file=f_train_source) for line in train_data_source]
[print(line, file=f_train_target) for line in train_data_target]
[print(line, file=f_val_source) for line in val_data_source]
[print(line, file=f_val_target) for line in val_data_target]
[print(line, file=f_test_source) for line in test_data_source]
[print(line, file=f_test_target) for line in test_data_target]

# close the input source and target files
fs.close()
ft.close()

# close the output files
f_train_source.close()
f_train_target.close()
f_val_source.close()
f_val_target.close()
f_test_source.close()
f_test_target.close()

print('fs_lines = ' + str(f_lines))
print('train_lines = ' + str(train_lines))
print('val_lines = ' + str(val_lines))
print('test_lines = ' + str(test_lines))
print('done')

1875
1875
1875
1875
fs_lines = 2206
train_lines = 1875
val_lines = 330
test_lines = 1
done


<h3> Make Vocabulary Inputs

In [65]:
for file in [filename_source, filename_target]:
# for file in [filename_source]:

    # open output files in subdirectory of input files directory (must create manually)
    fs = open(os.path.join(output_directory, file), 'r')

    # read the source and target input files line by line, stripping all whitespace and empty lines
    source_data = fs.read().strip().split('\n')

    # set min and max initial values
    source_data_min = float('Inf')
    source_data_max = 0.0

    for i in range(len(source_data)):
        # print([x for x in source_data[i].split(' ')])
        source_array = np.array([int(x[1:]) for x in source_data[i].split(' ')])
        if source_array.max() > source_data_max:
            source_data_max = source_array.max()
        if np.min(source_array[np.nonzero(source_array)]) < source_data_min:
            source_data_min = np.min(source_array[np.nonzero(source_array)])

    # print range of integers from min to max found in files
    range_size = (source_data_max - source_data_min) + 1
    samples = np.linspace(source_data_min, source_data_max, num=range_size, endpoint=True, retstep=False, dtype=int)
    # print(samples)

    samples_prefixed = []
    prefixes = ['a', 'b', 'c']
    [[samples_prefixed.append(prefix + str(x)) for x in samples] for prefix in prefixes]
    print(samples_prefixed)

    # save vocabulary input files to train_dir
    filename_noext, _ = os.path.splitext(file)
    np.savetxt(os.path.join(train_dir, filename_noext + '_vocab_input.txt'), samples_prefixed, delimiter=' ', fmt='%s')

# I can't limit the vocab as I need one embedding for each value in the source and the target files. Doing a linspace
#  creates additional values, but doesn't eliminate the existing ones.  
# for file in [filename_target]:
# 
#     # set min and max f0 values for target vocab (output contours)
#     source_data_min = 50
#     source_data_max = 550
# 
#     # print range of integers from min to max found in files
#     range_size = (source_data_max - source_data_min) + 1
#     samples = np.linspace(source_data_min, source_data_max, num=range_size, endpoint=True, retstep=False, dtype=int)
#     print(samples)
# 
#     # save vocabulary input files to train_dir
#     filename_noext, _ = os.path.splitext(file)
#     np.savetxt(os.path.join(train_dir, filename_noext + '_vocab_input.txt'), samples, delimiter=' ', fmt='%u')

# delete the input source and target files
# os.remove(os.path.join(output_directory, filename_source))
# os.remove(os.path.join(output_directory, filename_target))

# now run the vocabulary script to make the proper vocab files

['a50', 'a51', 'a52', 'a53', 'a54', 'a55', 'a56', 'a57', 'a58', 'a59', 'a60', 'a61', 'a62', 'a63', 'a64', 'a65', 'a66', 'a67', 'a68', 'a69', 'a70', 'a71', 'a72', 'a73', 'a74', 'a75', 'a76', 'a77', 'a78', 'a79', 'a80', 'a81', 'a82', 'a83', 'a84', 'a85', 'a86', 'a87', 'a88', 'a89', 'a90', 'a91', 'a92', 'a93', 'a94', 'a95', 'a96', 'a97', 'a98', 'a99', 'a100', 'a101', 'a102', 'a103', 'a104', 'a105', 'a106', 'a107', 'a108', 'a109', 'a110', 'a111', 'a112', 'a113', 'a114', 'a115', 'a116', 'a117', 'a118', 'a119', 'a120', 'a121', 'a122', 'a123', 'a124', 'a125', 'a126', 'a127', 'a128', 'a129', 'a130', 'a131', 'a132', 'a133', 'a134', 'a135', 'a136', 'a137', 'a138', 'a139', 'a140', 'a141', 'a142', 'a143', 'a144', 'a145', 'a146', 'a147', 'a148', 'a149', 'a150', 'a151', 'a152', 'a153', 'a154', 'a155', 'a156', 'a157', 'a158', 'a159', 'a160', 'a161', 'a162', 'a163', 'a164', 'a165', 'a166', 'a167', 'a168', 'a169', 'a170', 'a171', 'a172', 'a173', 'a174', 'a175', 'a176', 'a177', 'a178', 'a179', 'a180', '